In [5]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Joke about {topic} in a few sentences.")

model = ChatOllama(model="deepseek-r1:14b")

chain = prompt | model | StrOutputParser()

print(chain.invoke({"topic": "Japan"}))

<think>
Okay, so I need to come up with a joke about Japan. Hmm, where do I start? Well, Japan is known for a lot of things—sushi, samurai, karaoke, technology like Sony and Nintendo, hot springs, and their unique culture. Maybe I can play on one of these aspects.

I remember the previous joke mentioned sumo wrestlers and diet. That was funny because it's unexpected to hear about wrestlers being diet-conscious. So maybe I should think along those lines—something surprising or a twist on a common stereotype.

Let me think about other Japanese stereotypes. They're very polite, they bow a lot, use honorifics, have strict work ethics, love technology, and have a unique sense of fashion like Harajuku. Maybe something with that?

Wait, another thought: Japan is known for efficiency and punctuality. Trains run on time, people are准时, so maybe a joke about being late in Japan? Or perhaps something about their trains being too punctual.

Or maybe play on technology—like how they have the latest 

Local에서 쓰기에 deepseek-r1:70b 너무 느리다.     
deepseek-r1:14b 모델의 경우 위 파이프라인에 따른 질문에 대한 답에 5분 1초 가량 소요되었다.     
일본에 대한 농담은 거의 13분을 고민해서 답을 내놨다. CoT 과정을 다 보여준다.

In [2]:
# from langchain.vectorstores import Chroma
from langchain_chroma import Chroma

Chroma().delete_collection()

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_ollama import ChatOllama
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import warnings 
warnings.filterwarnings("ignore")

loader = PyPDFLoader(r"./data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# vectorstore = Chroma.from_documents(docs, embedding)
# ChromaDB 초기화
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding)

# batch_size를 줄여서 데이터 추가 (메모리 절약)
batch_size = 5  
for i in range(0, len(docs), batch_size):
    vectorstore.add_documents(docs[i:i + batch_size])
    print(f"{i + batch_size}개 문서 처리 완료") 
    
retriever = vectorstore.as_retriever()

# llm = ChatOllama(model="EEVE-Korean-10.8B:latest")

model = ChatOllama(model="deepseek-r1:14b")

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

5개 문서 처리 완료
10개 문서 처리 완료
15개 문서 처리 완료
20개 문서 처리 완료
25개 문서 처리 완료
30개 문서 처리 완료
35개 문서 처리 완료
40개 문서 처리 완료
45개 문서 처리 완료
50개 문서 처리 완료
55개 문서 처리 완료


In [2]:
for chunk in rag_chain.stream("국회의원의 임기는 뭐야?"):
    print(chunk, end="", flush=True)

<think>
Okay, I need to figure out the answer to the question "국회의원의 임기는 뭐야?" which means "What is the term of a member of parliament in South Korea?" 

Looking at the context provided, I see several articles. Article 42 clearly states that the term is four years. Other articles talk about other aspects like disqualifications, duties, and procedures for election, but none mention changes to the term length.

So, based on this, the answer should be straightforward—each MP serves a four-year term.
</think>

국회의원의 임기는 4년으로 규정되어 있습니다.